# COVID-19 Tracking U.S. Cases
> Tracking coronavirus total cases, deaths and new cases

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive, usa]
- image: images/covid-overview-us.png
- permalink: /covid-overview-us/

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/united-states.html
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/united-states.html



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_19-covid-{name}.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Province/State'   
# Confirmed, Recovered, Deaths
US_POI = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
    'Colorado', 'Connecticut', 'Delaware', 'Diamond Princess',
    'District of Columbia', 'Florida', 'Georgia', 'Grand Princess',
    'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
    'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
    'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
    'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
    'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
    'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
    'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
    'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming']

filter_us = lambda d: d[d['Country/Region'].eq('US') & d['Province/State'].isin(US_POI)]

df = get_frame('Confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df.pipe(filter_us)
dft_deaths = get_frame('Deaths').pipe(filter_us)
dft_recovered = get_frame('Recovered').pipe(filter_us)

LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]

dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfc_recovered = dft_recovered.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()
dfp_recovered = dft_recovered.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
for c in 'Cases, Deaths, Recovered'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table.head(15)

,Province/State,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate
0,New York,5365,34,0,525,2,0,4840,32,0,0.6
1,Washington,1376,74,0,572,37,1,804,37,0,5.4
2,California,952,18,0,340,5,6,612,13,0,1.9
3,New Jersey,742,9,0,69,1,0,673,8,0,1.2
4,Illinois,422,4,0,64,0,2,358,4,0,0.9
5,Florida,417,9,0,76,3,0,341,6,0,2.2
6,Louisiana,392,10,0,77,1,0,315,9,0,2.6
7,Michigan,334,3,0,25,0,0,309,3,0,0.9
8,Massachusetts,328,0,0,138,0,1,190,0,0,0.0
9,Georgia,287,10,0,66,1,0,221,9,0,3.5


In [6]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Recovered', 'Cases (+)', 'Deaths (+)', 'Recovered (+)']
s_ny = df_table[df_table[COL_REGION].eq('New York')][metrics].sum().add_prefix('NY ')
s_wa = df_table[df_table[COL_REGION].eq('Washington')][metrics].sum().add_prefix('WA ')
s_ca = df_table[df_table[COL_REGION].eq('California')][metrics].sum().add_prefix('CA ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_ny, **s_wa, **s_ca}
summary

{'updated': Timestamp('2020-03-19 00:00:00'),
 'since': Timestamp('2020-03-14 00:00:00'),
 'Cases': 13677,
 'Deaths': 200,
 'Recovered': 0,
 'Cases (+)': 10950,
 'Deaths (+)': 146,
 'Recovered (+)': 0,
 'NY Cases': 5365,
 'NY Deaths': 34,
 'NY Recovered': 0,
 'NY Cases (+)': 4840,
 'NY Deaths (+)': 32,
 'NY Recovered (+)': 0,
 'WA Cases': 1376,
 'WA Deaths': 74,
 'WA Recovered': 0,
 'WA Cases (+)': 804,
 'WA Deaths (+)': 37,
 'WA Recovered (+)': 0,
 'CA Cases': 952,
 'CA Deaths': 18,
 'CA Recovered': 0,
 'CA Cases (+)': 612,
 'CA Deaths (+)': 13,
 'CA Recovered (+)': 0}

In [7]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20
Province/State,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5,1,6,17,10,7,32
Alaska,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,2,3,3
Arizona,0,0,0,0,0,0,0,0,0,0,...,6,3,0,0,3,1,5,2,7,18
Arkansas,0,0,0,0,0,0,0,0,0,0,...,0,1,5,0,6,4,6,0,11,29
California,0,0,0,0,0,0,0,0,0,0,...,144,33,44,61,58,86,131,141,53,201


In [8]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-15]:dt_cols[LAST_DATE_I]],
    COL_REGION=COL_REGION,
    KPI_CASE='US',
    KPIS_INFO=[
        {'title': 'New York', 'prefix': 'NY'},
        {'title': 'Washington', 'prefix': 'WA'},
        {'title': 'California', 'prefix': 'CA'}],
    LEGEND_DOMAIN=[5, 50, 500, np.inf],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Interactive by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview-us.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/united-states.html)